In [18]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import re
import pickle
import string

In [ ]:
cred = credentials.Certificate("semaphore-a7fd1-firebase-adminsdk-dyeb3-e430d9430a.json")

In [2]:
app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [3]:
doc_ref = db.collection(u'posts')
docs = doc_ref.stream()
posts = []

for doc in docs:
    posts.append(doc.to_dict())

In [5]:
posts_df = pd.DataFrame(posts)

In [6]:
posts_df

,text,userImg,id,timestamp,username,tag
0,My phone has slowed down so much...,https://lh3.googleusercontent.com/a/AEdFTp6sqF...,CYIMUn78OTWEsb9iDhWS,2023-01-05 16:03:55.595000+00:00,Zack Zomor,zackzomor
1,"Gruppa Khmuri is a really good band, would rea...",https://firebasestorage.googleapis.com/v0/b/se...,j3yzggbxE9FCb4QTIHZR,2023-01-05 16:13:45.643000+00:00,Rasp Neri,raspneri
2,"Lorem ipsum dolor sit amet, consectetur adipis...",https://lh3.googleusercontent.com/a/AEdFTp6ezs...,gjNJo321L4j7LRA79xYD,2022-12-26 16:19:48.032000+00:00,Jesse Dixon,jessedixon
3,Biden is not that great actually,https://lh3.googleusercontent.com/a/AEdFTp6sqF...,CYIMUn78OTWEsb9iDhWS,2023-01-05 16:14:46.499000+00:00,Zack Zomor,zackzomor
4,Test,https://lh3.googleusercontent.com/a/AEdFTp6ezs...,gjNJo321L4j7LRA79xYD,2022-12-26 17:47:47.769000+00:00,Jesse Dixon,jessedixon
5,"in the end, I'm thankful we have Biden instead...",https://firebasestorage.googleapis.com/v0/b/se...,j3yzggbxE9FCb4QTIHZR,2023-01-05 16:21:41.614000+00:00,Rasp Neri,raspneri
6,The most dangerous thing in the world is a bad...,https://lh3.googleusercontent.com/a/AEdFTp6sqF...,CYIMUn78OTWEsb9iDhWS,2023-01-05 16:06:44.836000+00:00,Zack Zomor,zackzomor
7,Biden isn't that bad actually,https://firebasestorage.googleapis.com/v0/b/se...,j3yzggbxE9FCb4QTIHZR,2023-01-05 16:11:46.685000+00:00,Rasp Neri,raspneri
8,Eating healthy is really important,https://firebasestorage.googleapis.com/v0/b/se...,j3yzggbxE9FCb4QTIHZR,2023-01-05 16:13:23.774000+00:00,Rasp Neri,raspneri
9,Let's try this app out,https://lh3.googleusercontent.com/a/AEdFTp5pQU...,DQhPFAordF4XTa0FhNbt,2023-01-05 16:23:52.516000+00:00,Mel Summer,melsummer


In [11]:
posts_search= posts_df[posts_df.text.str.contains('Biden')]

In [12]:
posts_search

,text,userImg,id,timestamp,username,tag
3,Biden is not that great actually,https://lh3.googleusercontent.com/a/AEdFTp6sqF...,CYIMUn78OTWEsb9iDhWS,2023-01-05 16:14:46.499000+00:00,Zack Zomor,zackzomor
5,"in the end, I'm thankful we have Biden instead...",https://firebasestorage.googleapis.com/v0/b/se...,j3yzggbxE9FCb4QTIHZR,2023-01-05 16:21:41.614000+00:00,Rasp Neri,raspneri
7,Biden isn't that bad actually,https://firebasestorage.googleapis.com/v0/b/se...,j3yzggbxE9FCb4QTIHZR,2023-01-05 16:11:46.685000+00:00,Rasp Neri,raspneri


In [15]:
pickle_in = open("NB_classifier.pk1", 'rb')
NB_classifier = pickle.load(pickle_in)

In [16]:
def message_cleaning(message):
    Test_punc_removed = [char for char in message if char not in string.punctuation]
    Test_punc_removed_join = ''.join(Test_punc_removed)
    Test_punc_removed_join_clean = [word for word in Test_punc_removed_join.split() if word.lower() not in stopwords.words('english')]
    return Test_punc_removed_join_clean

In [19]:
posts_cleaned = posts_search['text'].apply(message_cleaning)

In [24]:
posts_cleaned.reshape(3, 47386)

AttributeError: 'Series' object has no attribute 'reshape'

In [21]:
vectorizer = CountVectorizer(analyzer = message_cleaning)
tweets_countvectorizer = CountVectorizer(analyzer = message_cleaning, dtype = 'uint8').fit_transform(posts_search['text']).toarray()

In [30]:
tweets_countvectorizer.shape

(3, 10)